Theres 917 actions in this demonstration  
So each feature is a 512 long vector for each frame

Each video has a feature (for each frame), and a groundtruth file that shows action at each step.   
Mapping is a file that has a number - action mapping


For Per : Feed in 1 data point, 1 ground truth and 1 mask. Mask is set to all true, CLS set to NONE  
```python
Calculating test metrics:
Segments shape: torch.Size([1, 13])
GT shape: torch.Size([1, 13])
Mask shape: torch.Size([1, 13])

Segments: tensor([[0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 2]], device='cuda:0')
GT: tensor([[1, 1, 1, 1, 1, 0, 0, 0, 2, 2, 2, 2, 2]], device='cuda:0')
Mask: tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True]], device='cuda:0')

CLS None
```

In [ ]:
from metrics import eval_mof, eval_f1, eval_miou, indep_eval_metrics, ClusteringMetrics
import numpy as np
import torch

In [32]:
segs = [
    [0, 0, 0, 1, 2, 2, 2],
    [0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2],
    [0, 0, 1, 1, 2, 2, 2, 2],
    [0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2],
    [0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 2],
    [0, 0, 1, 1, 2, 2, 2, 2],
    [0, 1, 2, 2, 2],
    [0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2],
    [0, 0, 0, 1, 1, 2, 2, 2],
    [0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2],
    [0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2],
    [0, 0, 1, 1, 1, 2, 2, 2, 2],
    [0, 1, 1, 2, 2, 2, 2, 2],
    [0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2],
    [0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2],
    [0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2]
]

segs = [torch.tensor(seg) for seg in segs]
np_segs = [seg.numpy() for seg in segs]

In [33]:
gt = [
    [0, 0, 0, 0, 1, 1, 2],
    [2, 2, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0],
    [2, 2, 0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 2, 2, 2, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 2],
    [1, 1, 0, 0, 0, 2, 2, 2],
    [0, 1, 2, 2, 2],
    [0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2],
    [0, 0, 0, 2, 2, 2, 2, 1],
    [2, 2, 2, 2, 2, 2, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2],
    [1, 1, 1, 1, 2, 0, 0, 0, 0],
    [0, 0, 0, 1, 2, 2, 2, 2],
    [0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2],
    [1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 0, 0, 0],
    [2, 2, 2, 2, 2, 2, 0, 1, 1, 1, 1]
]

gt = [torch.tensor(x) for x in gt]
np_gt = [x.numpy() for x in gt]
mask = [torch.ones_like(seg).bool() for seg in segs]

len(gt)

16

In [ ]:
# PER METRIC
results = indep_eval_metrics(
    segs, 
    gt,
    mask,
    metrics=['mof', 'f1', 'miou']
)

print(results)

{'mof': 0.8228394869019868, 'f1': 0.7399999949753333, 'miou': 0.6798611111111111}


In [ ]:
# FULL METRIC
mof_score, _ = eval_mof(
    np.concatenate(np_segs), 
    np.concatenate(np_gt),
    n_videos=len(segs)
)

f1_score, _ = eval_f1(
    np.concatenate(np_segs), 
    np.concatenate(np_gt),
    n_videos=len(segs)
)

miou_score, _ = eval_miou(
    np.concatenate(np_segs), 
    np.concatenate(np_gt),
    n_videos=len(segs)
)

print(f"MOF Score: {mof_score:.3f}")
print(f"F1 Score: {f1_score:.3f}")
print(f"mIoU Score: {miou_score:.3f}")

MOF Score: 0.465
F1 Score: 0.457
mIoU Score: 0.299


In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('tune.csv')

In [3]:
data.head()

,run_id,alpha-train,alpha-eval,lambda-frames-train,lambda-actions-train,lambda-frames-eval,lambda-actions-eval,eps-train,eps-eval,radius-gw,...,STD-Feats,State,Actions,Experiment,F1 Full,F1 Per,MIOU Full,MIOU Per,MOF Full,MOF Per
0,0,0.1,0.1,0.01,0.01,0.01,0.01,0.01,0.01,0.04,...,True,Simple,False,Standard Env;Random All,0.302648,0.622533,0.217812,0.521565,0.367367,0.687049
1,1,0.1,0.1,0.01,0.01,0.01,0.01,0.01,0.01,0.04,...,True,Simple,False,Standard Env;Random All,0.245105,0.557133,0.203142,0.519154,0.326326,0.644671
2,2,0.1,0.1,0.01,0.01,0.01,0.01,0.01,0.03,0.04,...,True,Simple,False,Standard Env;Random All,0.300070,0.623400,0.218226,0.530567,0.367367,0.688921
3,3,0.1,0.1,0.01,0.01,0.01,0.01,0.01,0.03,0.04,...,True,Simple,False,Standard Env;Random All,0.224406,0.557800,0.198732,0.523605,0.313313,0.633873
4,4,0.1,0.1,0.01,0.01,0.01,0.01,0.01,0.06,0.04,...,True,Simple,False,Standard Env;Random All,0.297491,0.646867,0.217256,0.548246,0.365365,0.703478


In [4]:
#Short by MOF Full
x = data.sort_values(by='MOF Per', ascending=False)
x.head()

,run_id,alpha-train,alpha-eval,lambda-frames-train,lambda-actions-train,lambda-frames-eval,lambda-actions-eval,eps-train,eps-eval,radius-gw,...,STD-Feats,State,Actions,Experiment,F1 Full,F1 Per,MIOU Full,MIOU Per,MOF Full,MOF Per
636,636,0.1,0.1,0.01,0.01,0.05,1.00,0.1,0.1,0.04,...,True,Simple,False,Standard Env;Random All,0.382718,0.862867,0.267152,0.838648,0.422422,0.916345
36,36,0.1,0.1,0.01,0.01,0.01,0.01,0.1,0.1,0.04,...,True,Simple,False,Standard Env;Random All,0.382718,0.862867,0.267152,0.838648,0.422422,0.916345
486,486,0.1,0.1,0.01,0.01,0.05,0.10,0.1,0.1,0.04,...,True,Simple,False,Standard Env;Random All,0.382718,0.862867,0.267152,0.838648,0.422422,0.916345
736,736,0.1,0.1,0.01,0.01,0.10,0.01,0.1,0.1,0.04,...,True,Simple,False,Standard Env;Random All,0.382718,0.862867,0.267152,0.838648,0.422422,0.916345
286,286,0.1,0.1,0.01,0.01,0.01,1.00,0.1,0.1,0.04,...,True,Simple,False,Standard Env;Random All,0.382718,0.862867,0.267152,0.838648,0.422422,0.916345


In [5]:
import itertools
import subprocess
import os
import csv
import re
from tqdm import tqdm
import random
import string


# Define ranges for hyperparameters to test
ALPHA_VALUES = [0.1, 0.3, 0.6, 1.0, 2.0]  # Example values for ALPHA_TRAIN and ALPHA_EVAL
LAMBDA_VALUES = [0.01, 0.05, 0.1, 0.5, 0.8, 1.0, 2.0]  # Example values for LAMBDA_*_TRAIN and LAMBDA_*_EVAL
EPS_VALUES = [0.01, 0.03, 0.06, 0.1, 1.0]  # Example values for EPS_TRAIN and EPS_EVAL
CLUSTERS = [3, 4]


# # Create all combinations of hyperparameters
combinations = itertools.product(ALPHA_VALUES, ALPHA_VALUES, \
                                LAMBDA_VALUES, LAMBDA_VALUES, LAMBDA_VALUES, LAMBDA_VALUES,\
                                EPS_VALUES, EPS_VALUES, CLUSTERS)

print(len(list(combinations)))

3001250
